In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Concatenate

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/dataset/base_consumption.csv")
df.head()

,Date,NR,WR,SR,ER,NER,TOTAL
0,1-Jan-15,835774,843214,708449,242186,34029,2663652
1,2-Jan-15,772043,872452,709401,250680,35225,2639801
2,3-Jan-15,853387,898691,718161,243188,34170,2747597
3,4-Jan-15,773651,842609,711702,214599,30153,2572714
4,5-Jan-15,798833,785888,716753,225151,31636,2558261


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    365 non-null    object
 1   NR      365 non-null    int64 
 2   WR      365 non-null    int64 
 3   SR      365 non-null    int64 
 4   ER      365 non-null    int64 
 5   NER     365 non-null    int64 
 6   TOTAL   365 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 20.1+ KB


In [ ]:
data = df[['NR', 'WR', 'SR', 'ER', 'NER', 'TOTAL']].values
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_data = scaler.fit_transform(data)

In [ ]:
# Split the data into features and target
X = data_normalized[:, :-1]  # Features (all columns except the last one)
y = data_normalized[:, -1]   # Target (last column)

In [ ]:
# Reshape X to [samples, time steps, features]
X = X.reshape(X.shape[0], 1, X.shape[1])

In [ ]:
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
#LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 64)                17920     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 17985 (70.25 KB)
Trainable params: 17985 (70.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
10/10 [==============================] - 3s 61ms/step - loss: 0.3438 - val_loss: 0.1732
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 0.2088 - val_loss: 0.0895
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 0.1140 - val_loss: 0.0363
Epoch 4/50
10/10 [==============================] - 0s 8ms/step - loss: 0.0550 - val_loss: 0.0104
Epoch 5/50
10/10 [==============================] - 0s 8ms/step - loss: 0.0266 - val_loss: 0.0058
Epoch 6/50
10/10 [==============================] - 0s 6ms/step - loss: 0.0196 - val_loss: 0.0084
Epoch 7/50
10/10 [==============================] - 0s 6ms/step - loss: 0.0195 - val_loss: 0.0090
Epoch 8/50
10/10 [==============================] - 0s 9ms/step - loss: 0.0187 - val_loss: 0.0077
Epoch 9/50
10/10 [==============================] - 0s 6ms/step - loss: 0.0178 - val_loss: 0.0068
Epoch 10/50
10/10 [==============================] - 0s 7ms/step - loss: 0.0172 - val_loss: 0.0063
Epoch 11/50
10/10 

In [ ]:
future_data = data_normalized[-1, :-1].reshape(1, 1, data_normalized.shape[1] - 1)  # Last row without target
future_prediction = np.array(model.predict(future_data))

# Example scaler initialization and fitting (for context)
# Assuming your original data has 5 features like in future_data's last print
# and you're predicting one feature (e.g., 'TOTAL')
scaler = MinMaxScaler()
# Fake data fitting for illustration; replace with your actual fitting data
scaler.fit(np.random.rand(100, 5))

# Given future_prediction
# future_prediction = np.array([[0.5274059]])
future_data = data_normalized[-1, :-1].reshape(1, 1, data_normalized.shape[1] - 1)  # Last row without target
future_prediction = np.array(model.predict(future_data))
# Your scaler expects the input to have the same number of features it was fitted on.
# Since you are dealing with a single feature prediction, create a dummy array
# that matches the scaler's expected input shape.
dummy_array = np.zeros((1, scaler.n_features_in_))

# Replace the first column of the dummy array with your prediction
# This assumes the feature you're predicting is the first feature the scaler was fitted on
dummy_array[:, 0] = future_prediction.flatten()

# Now inverse transform the dummy array
inverted_prediction = scaler.inverse_transform(dummy_array)

# Extract your actual prediction
actual_prediction = inverted_prediction[:, 0]

print(f"Original scaled prediction: {future_prediction.flatten()}")
print(f"Prediction in original units: {actual_prediction}")


1/1 [==============================] - 0s 20ms/step
Original scaled prediction: [0.5274059]
Prediction in original units: [0.53786738]


In [ ]:
future_data = data_normalized[-1, :-1].reshape(1, 1, data_normalized.shape[1] - 1)  # Last row without target
future_prediction = model.predict(future_data)
print (future_data)
print(future_prediction)

1/1 [==============================] - 0s 23ms/step
[[[0.45406899 0.62069119 0.43522199 0.68091619 0.68105782]]]
[[0.5274059]]


In [1]:
# future_prediction = future_prediction.reshape(1, 1, -1)
# # assert future_data.shape == (1, 1, 6)
# future_prediction = scaler.inverse_transform(future_prediction)
future_prediction = future_prediction.reshape(1, -1)  # Reshape to (1, 6)
assert future_prediction.shape == (1, 6)  # Check the shape

# Inverse transform the scaled prediction
future_prediction = scaler.inverse_transform(future_prediction)

NameError: name 'future_prediction' is not defined

In [ ]:
print('Predicted energy consumption for the next day:')
print(future_prediction)

Predicted energy consumption for the next day:


NameError: name 'future_prediction' is not defined